In [1]:
import numpy as np
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
from copy import copy
import random
import math
%matplotlib inline 

In [2]:
def generate_sets(size, players = None):
    def to_bin_vector(value):
        return tuple([(value >> i) & 1 for i in range(size)])
    
    if players is None:
        players = [i for i in range(size)]
        
    sets = []

    for n in range(2**size):
        combination = to_bin_vector(n)
        
        suitable = True
        
        for i in range(len(combination)):
            if combination[i] != 0 and i not in players:
                suitable = False
                break
                
        if suitable:
            sets.append(combination)
    
    return sets
                

In [3]:
def is_intersects(array1, array2):
    for i in range(len(array1)):
        if 1 == array1[i] == array2[i]:
            return True
    return False

def indexes_array(bitvector):
    return np.linspace(0, bitvector.shape[0] - 1, bitvector.shape[0], dtype = int)[bitvector]

def is_supper_additive(size, prices):
    all_combinations = generate_sets(size)

    for combination in all_combinations:
        comb_price = prices[combination]

        player_ids = indexes_array(np.array(combination) == 1)

        subcombinations = np.array(generate_sets(size, player_ids))

        subcombination_combinations = np.array(generate_sets(len(subcombinations)))

        for subcomb_comb in subcombination_combinations:
            current_combination = subcombinations[subcomb_comb == 1]

            if current_combination.shape[0] == 0:
                continue

            intersects = False

            for i in range(len(current_combination)):
                for j in range(len(current_combination)):
                    if i != j and is_intersects(current_combination[i], current_combination[j]):
                        intersects = True
                        break

                if intersects:
                    break
            if intersects:
                continue

            summary_price = 0

            for comb in current_combination:
                summary_price += prices[tuple(comb)]
                
            if summary_price > comb_price:
                return False, combination, [tuple(i) for i in current_combination]

    return (True, )

def is_convex(size, prices):
    all_combinations = generate_sets(size)

    for i in range(1, len(all_combinations)):
        for j in range(i + 1, len(all_combinations)):
            if i != j:
                intersection = [0 for k in range(size)]
                union = [0 for k in range(size)]
                for k in range(size):
                    if all_combinations[i][k] == 1 and all_combinations[j][k] == 1:
                        intersection[k] = 1
                    if all_combinations[i][k] == 1 or all_combinations[j][k] == 1:
                        union[k] = 1
                        
                intersection = tuple(intersection)
                union = tuple(union)
                        
                price1 = prices[union] + prices[intersection]
                price2 = prices[all_combinations[i]] + prices[all_combinations[j]]
                
                if price1 < price2:
                    print(price1, price2)
                    return (False, intersection, union, all_combinations[i], all_combinations[j])
                
    return True


def shaply_vector(size, prices, player):
    sets = generate_sets(size)
    
    summary = 0
    
    for S in sets:
        if S[player] != 0:
            S_weight = np.sum(S)

            # без и-го игрока
            S_i =list(S)
            S_i[player] = 0
            S_i = tuple(S_i)

            summary += math.factorial(np.sum(S) - 1) * math.factorial(size - np.sum(S)) * (prices[S] - prices[S_i])
            
    return (1/math.factorial(size)) * summary       
        

In [4]:
a1 = np.array([0, 1, 1])
a2 = np.array([0, 0, 1])
np.intersect1d(a1, a2)

array([0, 1])

In [5]:
prices_var15 = {
    (0, 0, 0, 0) : 0,
    (1, 0, 0, 0) : 2,
    (0, 1, 0, 0) : 3,
    (1, 1, 0, 0) : 6,
    (0, 0, 1, 0) : 2,
    (1, 0, 1, 0) : 5,
    (0, 1, 1, 0) : 6,
    (1, 1, 1, 0) : 9,
    (0, 0, 0, 1) : 4,
    (1, 0, 0, 1) : 7,
    (0, 1, 0, 1) : 8,
    (1, 1, 0, 1) : 11,
    (0, 0, 1, 1) : 8,
    (1, 0, 1, 1) : 10,
    (0, 1, 1, 1) : 12,
    (1, 1, 1, 1) : 14,
}

In [6]:
is_supper_additive(4, prices_var15)

(True,)

In [7]:
conv = is_convex(4, prices_var15)

print(conv)

for i in range(1, 5):
    print(prices_var15[conv[i]])
    
print()
if conv[0]:
    print('Is convex')
else:
    print('Not convex')

17 18
(False, (0, 1, 0, 0), (1, 1, 1, 1), (1, 1, 0, 0), (0, 1, 1, 1))
3
14
6
12

Not convex


In [8]:
vec = [shaply_vector(4, prices_var15, i) for i in range(4)]

print(np.round(vec, 3))
print(round(np.sum(vec), 3))

[2.417 3.75  2.917 4.917]
14.0
